In [37]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [38]:

import sys

import ipywidgets as widgets

sys.path.append('../')
sys.path.append('../src')

import src.backend
import src.workload_builder as builder
from mbi import Domain
import src.plots
import altair as alt
alt.renderers.enable('default')
from IPython.display import display, clear_output

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


alt.data_transformers.disable_max_rows()

/tmp/ipykernel_88776/218976982.py:16: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


DataTransformerRegistry.enable('default')

In [39]:
data_path = '../data/CPS/CPS.csv'
cps_domain = Domain(attrs=('age','income','marital','race'), shape=(100,100,7,4))
questions = ["Warm Up Question: Type 'Yes' if you are ready to begin!",
            "Question 1/6: How many Chinese and American Indians have 'Widowed' as their marital status?",
            "Question 2/6: How many people over the age of 65 have income between 100,000 and 149,000?",
            "Question 3/6: What is the second most common marital status of people with income in the 100,000 to 199,000 range?",
            "Question 4/6: Choose the top 3 age groups that have marital status: 'Never Married'?",
            "Question 5/6: In the 100,000 to 199,000 income range, rank the top 3 races in terms of count from greatest to least.",
            "Question 6/6: What is the second most common race in the 45-54 age group?"]

marital_race = {'marital':1, 'race':1}
age_income = {'age':10, 'income':10}
marital_income = {'income':10, 'marital':1}
age_marital = {'age':10, 'marital':1}
race_income = {'income':10, 'race':1}
age_race = {'age':10, 'race':1}

visualizations = [None, marital_race, age_income, marital_income, age_marital, race_income, age_race]
prev_spec = None
curr_spec = None
epsilon_increments = [None,0.05,0.05,0.05,0.05,0.05,0.05,0.05]
budget = 1.0
max_tries = 10

index = 0
tries = 0
back_end = None


# Logging

 ## Dropdown Boxes

Text Boxes

## Buttons

In [40]:
num_answers = len(questions)
survey_answers = [0]*num_answers

current_spec_arr = []
current_spec_remeasure = []
hist_workloads = []
histogram_plots = []
# hist_plot = None


def make_vis(do_not_update=None):
    global index, survey_answers, visualizations, epsilon_increments, curr_spec, back_end, bar_label, interface, ques
    back_end = src.backend.initialize_backend(cps_domain, data_path, budget=1.0)

    seed = 1234
    for i in range(len(questions) - 1):
        # prompt_arr[i+1].value = questions[i+1]
        hist = builder.histogram_workload(cps_domain.config, bin_widths=visualizations[i+1])
        hist_workloads.append(hist)

        y_hat, strategy_matrix = back_end.measure_hdmm(workload=hist, eps=epsilon_increments[i+1], restarts=20, seed=seed) # got y_hat

        column_names = list(visualizations[i+1].keys())
        curr_spec, cached = back_end.display(hist)
        current_spec_arr.append(curr_spec)
        current_spec_remeasure.append(curr_spec)
        plot = src.plots.linked_hist(column_names[0], column_names[1], data=curr_spec.reset_index(column_names), display_true=False, history=False)
        plot.save('charts/init_vis/q_' + str(i+1) + '.json')

def binning(measure_dict, i_id, j_id, epsilon=None):
    global tries, curr_spec, back_end, current_spec_arr, hist_workloads, plot_output_rm_arr

    left_col = measure_dict['left']
    right_col = measure_dict['right']

    widths = {left_col[0]:left_col[1], right_col[0]:right_col[1]}

    hist = builder.histogram_workload(cps_domain.config, bin_widths=widths)
    y_hat, strategy_matrix = back_end.measure_hdmm(workload=hist, eps=epsilon, restarts=20)

    prev_spec = current_spec_remeasure[i_id]

    # print("Spec Previous", prev_spec)
    # print("---------------------------------------------------------------")
    prev_spec.rename(columns={'error': 'error_prev', 'plus_error': 'plus_error_prev', 'minus_error': 'minus_error_prev', 'true_count':'true_count_prev', 'noisy_count':'noisy_count_prev'}, inplace=True)

    current_spec_remeasure[i_id], cached = back_end.display(hist) ################# Cache_search function called

    # print("spec after ", prev_spec)

    spec = current_spec_remeasure[i_id].join(prev_spec, on=[left_col[0], right_col[0]]).reset_index([left_col[0], right_col[0]])

    plot_new = src.plots.linked_hist_test(left_col[0], right_col[0], data=spec, projection=True,label=False)
    plot_new.save('charts/rm_q' + str(i_id+1) + '/rm_' + str(j_id+1) + '.json')

make_vis()

print("Init Vis Complete!")

for i in range(len(questions) - 1):
    key_val = [x for x in visualizations[i+1].items()]
    measure_dict = {'left': key_val[0], 'right': key_val[1]}
    for j in range(10):
        binning(measure_dict, i, j, epsilon = epsilon_increments[i+1])

print("Remeasure Complete!")

Init Vis Complete!
Remeasure Complete!
